In [49]:
#Notebook to re-run the EBM experiments (perturbations at all latitudes)
#with 0% relative humidity. 

#Adapted from EBM_Gregory_run_noG.ipynb

#18 December 2019: add instantaneous CO2 forcing run

#Note: in some cases runs completed in the background but output did not print because 
#window had been closed. 

In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd #necessary for new dimension in concat

In [3]:
#Perturbation version of the EBM with forcing, feedback, ocean heat uptake, and control temperature inputs
# 5-21-19: modified to only diffuse anomalous MSE rather than absolute. 

# Inputs:
# Rf: radiative forcing as a function of latitude
# lamda: feedback as a function of latitude (misspelled because "lambda" has special meaning in Python)
# G: ocean heat uptake (i.e. change in net surface heat fluxes) as a function of latitude
# T_control: temperature from the control climate (do things based on perturbations ot this)
# lats: latitudes for the model grid
# latBounds: can be specified, or automatically calculated based on given lats (put in default based on TRACMIP climatology common grid)
# anom: whether to diffuse anomalous rather than absolute MSE (papers and code from Aaron suggest it should be this way)
#
# Inputs with defalt values:
# D: Diffusion coefficient (m^2/s)
# dt: time step (arbitrary units)
# ps: surface pressure (Pa)
# RH: relative humidity (fraction)
# C: ocean heat capacity (arbitrary units)
# nmax: maximum number of iterations before giving up if equilibrium not reached
# tol: Max. temperature difference between 2 time steps to define equilibrium (K)
# tempPrint: if true, print the temperatures at selected latitudes when indicating progress
#
# Output: 2 XArray datasets, one on latitude and one on boundary latitude grid
def moistEBM_perturb(Rf, lamda, G, T_control, lats, latBounds='auto',
                  D=1.06e6, dt=1.0e-4, ps=9.8e4, RH=0.8, C=1, nmax=5e5, tol=0.001, tempPrint = False):
    
    #Physical constants
    Cp = 1004.   #Heat capacity of air at constant pressure (J/(kg*K))
    Lv = 2.5e6  #Latent heat vaporization at 0 deg C (J/kg)
    g = 9.8 #Gravitational acceleration (m/s^2)
    r_earth = 6.37e6 #Earth radius (m)
    Rd = 287. #Gas constant for dry air
    Rv = 461. #Gas constant for water vapor
    Lv = 2.5e6 #Latent heat of vaporization
    
    TS = T_control+0 #Initial temperature from control climate (+0 to avoid possibly changing T_control later)
    
    #Set up matrices for numerical integration of diffusion PDE
    #A and B represent transport of heat into grid box from the north and the south, respectively;
    #together they represent a 2nd-order finite-difference second derivative, weighted by latitude. 
    A = np.zeros((len(lats),len(lats)))
    B = np.zeros((len(lats),len(lats)))
    for i in range(1,len(lats)-1):
        A[i,i]=-1
        A[i,i+1]=1
        B[i,i]=1
        B[i,i-1]=-1
        
        
    #Calculate latitude boundaries if not specified
    if latBounds == 'auto':
        latBounds = np.append(-90, lats+(lats[1]-lats[0])/2.)
        
    #Define grid for EBM
    #print(latBounds)
    latDiff = np.diff(latBounds) #Grid spacing 
    latsNB = latBounds[1:len(latBounds)] #Latitudes at North Boundaries of grid boxes
    latsSB = latBounds[0:len(latBounds)-1] #Latitudes at South Boundaries of grid boxes
    lats_rad = lats*np.pi/180. #Convert from degrees to radians
    latsNB_rad = latsNB*np.pi/180.
    latsSB_rad = latsSB*np.pi/180.
    latDiff_rad = latDiff*np.pi/180.
    
    if tempPrint:
        #Print the temperatures at a few selected latitudes while running
        #Floor division to get integer indices
        printIndices=[len(lats)//10, 
                      len(lats)//5, 
                      len(lats)//3, 
                      len(lats)//2, 
                      (2*len(lats))//3, 
                      (4*len(lats))//5, 
                      (9*len(lats))//10]
        print('Latitudes for test temperature printing:')
        for k in range(len(printIndices)):
            print(lats[printIndices[k]])
    
    #Iterate through temperature tendency
    n = 0 #Iterative loop counter
    conv = False #Whether we've reached equilibrium
    while((n < nmax) & (conv == False)):
        #Set up profile of moist static energy
        #Saturation vapor pressure as a function of latitude:        
        es = 611.*np.exp((Lv/Rv)*((1./273.15)-(1./TS))) #Hartmann, 2016, eq. 1.11
        q = (RH*es)/ps*Rd/Rv #Specific humidity (kg/kg): RH = e/es therefore e = RH*es, and q = e/ps*Rd/Rv
        MSE = Cp*TS+Lv*q #Moist static energy at surface
        
        #Calculate climatological MSE and then perturbation by subtracting it out
        es_control = 611.*np.exp((Lv/Rv)*((1./273.15)-(1./T_control)))
        q_control = (RH*es_control)/ps*Rd/Rv
        MSE_control = Cp*T_control+Lv*q_control
        MSE_anom = MSE - MSE_control
         
        #Calculate diffusion/transport of anomalous MSE
        Transport = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad)*latDiff_rad*latDiff_rad))*(
                    np.cos(latsNB_rad)*np.dot(A,MSE_anom)-np.cos(latsSB_rad)*np.dot(B,MSE_anom))
        #Diffusion at south boundary
        Transport[0] = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad[0])*latDiff_rad[0]*latDiff_rad[0]))*(
                       np.cos(lats_rad[1])*(MSE_anom[2]-MSE_anom[1])-np.cos(lats_rad[0])*(MSE_anom[1]-MSE_anom[0]))
        #Diffusion at north boundary
        Transport[len(lats)-1] = (D*(ps/g)/(r_earth*r_earth*np.cos(lats_rad[len(lats)-1])*latDiff_rad[len(lats)-1]*latDiff_rad[len(lats)-1]))*(
                       np.cos(lats_rad[len(lats)-1])*(MSE_anom[len(lats)-1]-MSE_anom[len(lats)-2])-
                       np.cos(lats_rad[len(lats)-2])*(MSE_anom[len(lats)-2]-MSE_anom[len(lats)-3]))
        
        #Calculate perturbation temperature for feedback calculation (will be 0 on first time step)
        TS_perturb = TS - T_control
        
        #Temperature tendency/time stepping. When equilibrium is reached, the new and old TS cancel
        #and this equation represents energy balance, e.g. equation (1) of Bonan et al. (2018). 
        #Transport out
        TS = TS + dt*(Rf+lamda*TS_perturb+Transport-G)/C
        
        #Check for equilibrium (temperature has stopped changing) and return results if so
        #print(np.max(np.abs(EI-OLR+Transport)))
        if ((np.max(np.abs(Rf+lamda*TS_perturb+Transport-G)) < tol) and (n > 0)):
            conv = True
            print('EBM reached equilibrium in ' + str(n) + ' iterations')
            
            #Calculate the northward transport by integrating the transport/diffusion term northward
            NorthTransport = -np.cumsum(Transport*(2*np.pi*r_earth*r_earth)*(np.sin(latsNB_rad)-np.sin(latsSB_rad)))
            
            #Correct for transport at north pole (caused by numerics)
            N = NorthTransport[-1] #Northward energy transport at NP
            E = N/2.*(1+np.sin(lats*np.pi/180.)) 
            CorrectedTransport = NorthTransport - E
            
            #Versions with 0 appended (for plotting at boundary latitudes)
            NorthTransportBounds = np.append(0, NorthTransport)
            CorrectedTransportBounds = np.append(0, CorrectedTransport)
            
            #Return 2 XArray objects: ds_lat (for variables on the latitude grid) and ds_latBounds (for variables on boundary latitudes)
            ds_lat = xr.Dataset({'CorrectedTransport': (['lat'], CorrectedTransport),
                                 'Diffusion': (['lat'], Transport), 
                                 'MSE': (['lat'], MSE), 
                                 'MSE_anom': (['lat'], MSE_anom),
                                 'NorthTransport': (['lat'], NorthTransport), 
                                 'q': (['lat'], q), 
                                 'TS_perturb': ('lat', TS_perturb),
                                 'TS': (['lat'], TS)}, 
                                coords = {'lat': (['lat'], lats)})
            
#             ds_latBounds = xr.Dataset({'CorrectedTransportBounds': (['lat'], CorrectedTransportBounds), 
#                                        'NorthTransportBounds': (['lat'], NorthTransportBounds)}, 
#                                       coords ={'lat': (['lat'], latBounds)})
            
            return ds_lat
            
        if np.mod(n,6000)==0:
            print('EBM (perturbation) finished iteration ' + str(n))
            if tempPrint:
                print('{:e}, {:e}, {:e}, {:e}, {:e}, {:e}, {:e} K'.format(TS[printIndices[0]], 
                                                                          TS[printIndices[1]],
                                                                          TS[printIndices[2]],
                                                                          TS[printIndices[3]],
                                                                          TS[printIndices[4]],
                                                                          TS[printIndices[5]],
                                                                          TS[printIndices[6]]))
                
        n = n + 1 #Iterate loop counter
        #print(n)
        #Warning message if equilibrium not reached (Will return nothing)
        if(n == nmax):
            #print('WARNING: equilibrium not reached in moist EBM--nothing returned')
            print('WARNING: equilibrium not reached in moist EBM--nans returned')
            
            #Should really return all nans instead
            nanarray = np.ones(len(lats))*np.nan
            ds_lat = xr.Dataset({'CorrectedTransport': (['lat'], nanarray),
                                 'Diffusion': (['lat'], nanarray), 
                                 'MSE': (['lat'], nanarray), 
                                 'MSE_anom': (['lat'], nanarray),
                                 'NorthTransport': (['lat'], nanarray), 
                                 'q': (['lat'], nanarray), 
                                 'TS_perturb': ('lat', nanarray),
                                 'TS': (['lat'], nanarray)}, 
                                coords = {'lat': (['lat'], lats)})
            
            print(ds_lat)
            return ds_lat

In [5]:
#Load climatology data (for climatological temperature) and the Gregory regression output
ds_clim = xr.open_dataset('../nc_revised_20181130/master.nc')
ds_greg = xr.open_dataset('nc_from_xarray/GregoryTotalForcingFeedbackStorage_a4.nc')

In [6]:
models = ['AM2', 'CAM3', 'CAM4', 'CNRM-AM6-DIA-v2', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT',
       'NorESM2']

In [7]:
#CNRM model had "divide by zero" error for water vapor, so skip it for water vapor kernels
models_noCNRM = ['AM2', 'CAM3', 'CAM4', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT',
       'NorESM2']

In [8]:
#Skipping NorESM2 as well because it didn't have clear-sky LW output saved
models_10 = ['AM2', 'CAM3', 'CAM4', 'ECHAM-6.1', 'ECHAM-6.3',
       'IPSL-CM5A', 'MIROC5', 'MPAS', 'MetUM-GA6-CTL', 'MetUM-GA6-ENT']

In [9]:
lats = ds_clim.lat.data

In [10]:

#####   RUNS   #####


In [11]:

###   Control Runs (9.6*10^5 diffusivity)   ###


In [12]:
EBM_perturb_results_096e4_dict = dict()
for model in models:
    print('Running perturbation EBM (Aqua4xCO2) for model: ' + model)
    EBM_perturb_results_096e4_dict[model] = moistEBM_perturb(ds_greg['forcing'].sel(model=model).data, 
                                            ds_greg['feedback'].sel(model=model).data,
                                            0, 
                                            ds_clim['ts'].sel(model=model, exp='AquaControl').mean(dim='lon').mean(dim='time').data,
                                            ds_clim.lat.data, D=0.96e6, dt=5.0e-5, nmax=8e5, RH=0)

Running perturbation EBM (Aqua4xCO2) for model: AM2
EBM (perturbation) finished iteration 0
EBM (perturbation) finished iteration 6000
EBM (perturbation) finished iteration 12000
EBM (perturbation) finished iteration 18000
EBM (perturbation) finished iteration 24000
EBM (perturbation) finished iteration 30000
EBM (perturbation) finished iteration 36000
EBM (perturbation) finished iteration 42000
EBM (perturbation) finished iteration 48000
EBM (perturbation) finished iteration 54000
EBM (perturbation) finished iteration 60000
EBM (perturbation) finished iteration 66000
EBM (perturbation) finished iteration 72000
EBM (perturbation) finished iteration 78000
EBM (perturbation) finished iteration 84000
EBM (perturbation) finished iteration 90000
EBM (perturbation) finished iteration 96000
EBM (perturbation) finished iteration 102000
EBM (perturbation) finished iteration 108000
EBM (perturbation) finished iteration 114000
EBM (perturbation) finished iteration 120000
EBM (perturbation) finish

In [13]:
#Concatenate and save
EBM_perturb_results_096e4 = xr.concat(list(EBM_perturb_results_096e4_dict.values())[:], dim=pd.Index(models, name='model')) 
EBM_perturb_results_096e4.to_netcdf('nc_from_xarray/EBM_perturb_results_noQ_096e4.nc')

In [14]:

###   Perturbing Rapid Adjustments And Feedbacks One At A Time   ###


In [15]:
#8-8-19: Having calculated individual rapid adjustments and feedbacks using Gregory regressions, 
#time to perturb them one at a time in the EBM. 
#The perturbations are done as subtractions from the forcing and feedback terms in the control runs above. 

#Naming convention: 
#EBM_results_[adj, fb]_[SW, LW]_[cloud, noncloud, ta, wv, ts]

In [16]:
#Load the results and subset to individual adjustments/feedbacks
#Flip sign for LW feedbacks because kernels are positive upward
ds_SW = xr.open_dataset('nc_from_xarray/GregoryAPRP_a4.nc')
ds_LW_ta = xr.open_dataset('nc_from_xarray/Gregory_kernel_ta_a4.nc')
ds_LW_wv = xr.open_dataset('nc_from_xarray/Gregory_kernel_wv_a4.nc')
ds_LW_ts = xr.open_dataset('nc_from_xarray/Gregory_kernel_ts_a4.nc')
ds_LW_cloud = xr.open_dataset('nc_from_xarray/Gregory_dLWCRE_co_a4.nc')

In [17]:
da_adj_SW_cloud = ds_SW['adj_cloud']
da_adj_SW_noncloud = ds_SW['adj_noncloud']
da_adj_LW_ta = -ds_LW_ta['adj']
da_adj_LW_wv = -ds_LW_wv['adj']
da_adj_LW_cloud = ds_LW_cloud['adj']

In [18]:
da_fb_SW_cloud = ds_SW['fb_cloud']
da_fb_SW_noncloud = ds_SW['fb_noncloud']
da_fb_LW_ta = -ds_LW_ta['fb']
da_fb_LW_wv = -ds_LW_wv['fb']
da_fb_LW_ts = -ds_LW_ts['fb'] #No rapid adjustment for this one
da_fb_LW_cloud = ds_LW_cloud['fb']

In [19]:
#The surface temperature kernel has nans at the edges--duplicate the adjacent values
#to avoid the EBM returning nans
#print(da_fb_LW_ts)
da_fb_LW_ts.loc[dict(lat=-89.5)] = da_fb_LW_ts.loc[dict(lat=-88.5)]
da_fb_LW_ts.loc[dict(lat=89.5)] = da_fb_LW_ts.loc[dict(lat=88.5)]
#print(da_fb_LW_ts)

In [20]:
#Other kernels had weird edge effects at outer 1-2 latitudes; 
#may need to fix this but shouldn't matter much

In [21]:
#Fix edge effects for LW cloud stuff as well

In [22]:
for model in models_10:
    if model in ['CAM3']:
        da_adj_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_adj_LW_cloud.sel(model=model, lat=-87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=-88.5)] = da_adj_LW_cloud.sel(model=model, lat=-87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=89.5)] = da_adj_LW_cloud.sel(model=model, lat=87.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=88.5)] = da_adj_LW_cloud.sel(model=model, lat=87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_fb_LW_cloud.sel(model=model, lat=-87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-88.5)] = da_fb_LW_cloud.sel(model=model, lat=-87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=89.5)] = da_fb_LW_cloud.sel(model=model, lat=87.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=88.5)] = da_fb_LW_cloud.sel(model=model, lat=87.5)
    else:
        da_adj_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_adj_LW_cloud.sel(model=model, lat=-88.5)
        da_adj_LW_cloud.loc[dict(model=model, lat=89.5)] = da_adj_LW_cloud.sel(model=model, lat=88.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=-89.5)] = da_fb_LW_cloud.sel(model=model, lat=-88.5)
        da_fb_LW_cloud.loc[dict(model=model, lat=89.5)] = da_fb_LW_cloud.sel(model=model, lat=88.5)
        

In [50]:
#Calculate the instantaneous rapid adjustment (as residual) and fix edge lats if necessary
f_inst = ds_greg['forcing'] - (ds_SW['adj_cloud'] + ds_SW['adj_noncloud']
                                + ds_LW_cloud['adj'] - ds_LW_wv['adj'] - ds_LW_ta['adj'])
#print(f_inst.isel(lat=0)) #OK, no edge zeros
#print(f_inst) #DataArray--should be fine

In [23]:
#Functions to run the EBM for each model for a given process
#(call with G = 0 here--will fix that problem)
#These versions setting RH = 0, hard-coded for this notebook.

#One for adjustment, one for feedback, possibly one for both
#ds_clim: for climatological temperature
#ds_control: Dataset containing forcing, feedback terms
#da_perturb: DataArray loaded above for the rapid adjustments/feedbacks
#D: diffusivity
#dt, nmax: time step, max number of iterations
#Can enter dt, nmax as dicts, different for each model, instead.
#Try-except since don't want to break backward compatibility
#scale: percentage of the perturbation to apply (default 1 for backward compatibility)
#
#Returns: EBM_results_da: XArray DataArray containing the results for each model. 
#(save to NetCDF separately)
def run_EBM_perturb_adj(ds_clim, ds_control, da_perturb, models, D, dt=9.0e-5, nmax=7e5, scale=1, tempPrint=True):
    EBM_results_dict = dict()
    for model in models: 
        print(model)
        try:
            dt_model = dt[model] #input dict
        except:
            dt_model = dt
        try: 
            nmax_model = nmax[model]
        except: 
            nmax_model = nmax
        EBM_results_dict[model] = moistEBM_perturb(
            ds_control['forcing'].sel(model=model).data - da_perturb.sel(model=model).data * scale, 
            ds_control['feedback'].sel(model=model).data,
            0,
            ds_clim['ts'].sel(model=model, exp='AquaControl').mean(dim='lon').mean(dim='time').data,
            ds_clim.lat.data, 
            D=D, dt=dt_model, nmax=nmax_model, tempPrint=tempPrint, RH = 0 
            )
    EBM_results_da = xr.concat(list(EBM_results_dict.values())[:], dim=pd.Index(models, name='model'))
    return EBM_results_da
    
def run_EBM_perturb_fb(ds_clim, ds_control, da_perturb, models, D, dt=9.0e-5, nmax=7e5, scale=1, tempPrint=True):
    EBM_results_dict = dict()
    for model in models:
        print(model)
        try:
            dt_model = dt[model] #input dict
        except:
            dt_model = dt
        try: 
            nmax_model = nmax[model]
        except: 
            nmax_model = nmax
        EBM_results_dict[model] = moistEBM_perturb(
            ds_control['forcing'].sel(model=model).data, 
            ds_control['feedback'].sel(model=model).data - da_perturb.sel(model=model).data * scale,
            0,
            ds_clim['ts'].sel(model=model, exp='AquaControl').mean(dim='lon').mean(dim='time').data,
            ds_clim.lat.data, 
            D=D, dt=dt_model, nmax=nmax_model, tempPrint=tempPrint, RH = 0 
            )
    EBM_results_da = xr.concat(list(EBM_results_dict.values())[:], dim=pd.Index(models, name='model'))
    return EBM_results_da

In [24]:
#Run for each case and save to NetCDF files


In [51]:
EBM_results_f_inst = run_EBM_perturb_adj(ds_clim, ds_greg, f_inst, models_10, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841678e+02, 2.896704e+02, 3.004598e+02, 3.080435e+02, 3.055914e+02, 2.953126e+02, 2.908842e+02 K
EBM (perturbation) finished iteration 6000
2.839621e+02, 2.898894e+02, 3.006544e+02, 3.081533e+02, 3.059247e+02, 2.954428e+02, 2.901159e+02 K
EBM (perturbation) finished iteration 12000
2.840072e+02, 2.899517e+02, 3.007173e+02, 3.082093e+02, 3.060203e+02, 2.954514e+02, 2.900148e+02 K
EBM (perturbation) finished iteration 18000
2.840478e+02, 2.899941e+02, 3.007587e+02, 3.082396e+02, 3.060562e+02, 2.954680e+02, 2.900117e+02 K
EBM (perturbation) finished iteration 24000
2.840773e+02, 2.900234e+02, 3.007849e+02, 3.082560e+02, 3.060755e+02, 2.954844e+02, 2.900250e+02 K
EBM (perturbation) finished iteration 30000
2.840974e+02, 2.900429e+02, 3.008014e+02, 3.082659e+02, 3.060880e+02, 2.954977e+02, 2.900388e+02 K
EBM (perturbation) finished iteration 36000
2.841106e+02, 2.90055

In [52]:
EBM_results_f_inst.to_netcdf('nc_from_xarray/EBM_results_noQ_f_inst.nc')

In [25]:
EBM_results_adj_SW_cloud = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_SW_cloud, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841687e+02, 2.896709e+02, 3.004602e+02, 3.080441e+02, 3.055918e+02, 2.953131e+02, 2.908852e+02 K
EBM (perturbation) finished iteration 6000
2.873637e+02, 2.926311e+02, 3.029604e+02, 3.102536e+02, 3.081613e+02, 2.982605e+02, 2.942856e+02 K
EBM (perturbation) finished iteration 12000
2.894240e+02, 2.946176e+02, 3.046750e+02, 3.114112e+02, 3.099884e+02, 3.005000e+02, 2.968281e+02 K
EBM (perturbation) finished iteration 18000
2.907893e+02, 2.959351e+02, 3.058021e+02, 3.121673e+02, 3.112501e+02, 3.021202e+02, 2.987049e+02 K
EBM (perturbation) finished iteration 24000
2.916923e+02, 2.968054e+02, 3.065460e+02, 3.126743e+02, 3.121299e+02, 3.032753e+02, 3.000576e+02 K
EBM (perturbation) finished iteration 30000
2.922892e+02, 2.973811e+02, 3.070400e+02, 3.130180e+02, 3.127451e+02, 3.040932e+02, 3.010208e+02 K
EBM (perturbation) finished iteration 36000
2.926845e+02, 2.97763

In [26]:
EBM_results_adj_SW_cloud.to_netcdf('nc_from_xarray/EBM_results_noQ_adj_sw_cloud.nc')

In [27]:
EBM_results_adj_SW_noncloud = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_SW_noncloud, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896706e+02, 3.004602e+02, 3.080439e+02, 3.055918e+02, 2.953128e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.855567e+02, 2.915475e+02, 3.029501e+02, 3.100959e+02, 3.078930e+02, 2.971238e+02, 2.917843e+02 K
EBM (perturbation) finished iteration 12000
2.869114e+02, 2.930063e+02, 3.044143e+02, 3.111296e+02, 3.093020e+02, 2.985536e+02, 2.931854e+02 K
EBM (perturbation) finished iteration 18000
2.879229e+02, 2.940193e+02, 3.053307e+02, 3.117458e+02, 3.102277e+02, 2.996420e+02, 2.943831e+02 K
EBM (perturbation) finished iteration 24000
2.886191e+02, 2.946986e+02, 3.059206e+02, 3.121392e+02, 3.108613e+02, 3.004379e+02, 2.952944e+02 K
EBM (perturbation) finished iteration 30000
2.890847e+02, 2.951487e+02, 3.063065e+02, 3.123995e+02, 3.113018e+02, 3.010092e+02, 2.959596e+02 K
EBM (perturbation) finished iteration 36000
2.893932e+02, 2.95446

In [28]:
EBM_results_adj_SW_noncloud.to_netcdf('nc_from_xarray/EBM_results_noQ_adj_sw_noncloud.nc')

In [29]:
EBM_results_adj_LW_ta = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_ta, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896705e+02, 3.004601e+02, 3.080439e+02, 3.055918e+02, 2.953128e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.853616e+02, 2.911991e+02, 3.026591e+02, 3.100201e+02, 3.077986e+02, 2.970673e+02, 2.919975e+02 K
EBM (perturbation) finished iteration 12000
2.865248e+02, 2.924708e+02, 3.039744e+02, 3.109825e+02, 3.091525e+02, 2.984795e+02, 2.934167e+02 K
EBM (perturbation) finished iteration 18000
2.874120e+02, 2.933670e+02, 3.047989e+02, 3.115543e+02, 3.100437e+02, 2.995450e+02, 2.945995e+02 K
EBM (perturbation) finished iteration 24000
2.880295e+02, 2.939727e+02, 3.053315e+02, 3.119197e+02, 3.106527e+02, 3.003190e+02, 2.954903e+02 K
EBM (perturbation) finished iteration 30000
2.884457e+02, 2.943767e+02, 3.056814e+02, 3.121619e+02, 3.110751e+02, 3.008720e+02, 2.961364e+02 K
EBM (perturbation) finished iteration 36000
2.887232e+02, 2.94645

In [30]:
EBM_results_adj_LW_ta.to_netcdf('nc_from_xarray/EBM_results_noQ_adj_lw_ta.nc')

In [31]:
EBM_results_adj_LW_wv = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_wv, models_noCNRM, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080439e+02, 3.055918e+02, 2.953128e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.857531e+02, 2.918362e+02, 3.034740e+02, 3.104181e+02, 3.080236e+02, 2.971757e+02, 2.917590e+02 K
EBM (perturbation) finished iteration 12000
2.873423e+02, 2.935633e+02, 3.052208e+02, 3.116155e+02, 3.095486e+02, 2.986773e+02, 2.932085e+02 K
EBM (perturbation) finished iteration 18000
2.885382e+02, 2.947616e+02, 3.063002e+02, 3.123212e+02, 3.105577e+02, 2.998383e+02, 2.944734e+02 K
EBM (perturbation) finished iteration 24000
2.893600e+02, 2.955616e+02, 3.069899e+02, 3.127689e+02, 3.112504e+02, 3.006952e+02, 2.954482e+02 K
EBM (perturbation) finished iteration 30000
2.899073e+02, 2.960893e+02, 3.074386e+02, 3.130639e+02, 3.117331e+02, 3.013141e+02, 2.961656e+02 K
EBM (perturbation) finished iteration 36000
2.902683e+02, 2.96436

In [32]:
EBM_results_adj_LW_wv.to_netcdf('nc_from_xarray/EBM_results_noQ_adj_lw_wv.nc')

In [33]:
EBM_results_fb_SW_cloud = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_SW_cloud, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.854174e+02, 2.912326e+02, 3.026476e+02, 3.101401e+02, 3.077495e+02, 2.968361e+02, 2.916941e+02 K
EBM (perturbation) finished iteration 12000
2.860925e+02, 2.919440e+02, 3.035390e+02, 3.109732e+02, 3.087171e+02, 2.975583e+02, 2.923144e+02 K
EBM (perturbation) finished iteration 18000
2.864026e+02, 2.922590e+02, 3.039188e+02, 3.113293e+02, 3.091496e+02, 2.979023e+02, 2.926282e+02 K
EBM (perturbation) finished iteration 24000
2.865396e+02, 2.923966e+02, 3.040826e+02, 3.114840e+02, 3.093432e+02, 2.980606e+02, 2.927758e+02 K
EBM (perturbation) finished iteration 30000
2.865994e+02, 2.924566e+02, 3.041538e+02, 3.115518e+02, 3.094296e+02, 2.981324e+02, 2.928433e+02 K
EBM (perturbation) finished iteration 36000
2.866255e+02, 2.92482

In [34]:
EBM_results_fb_SW_cloud.to_netcdf('nc_from_xarray/EBM_results_noQ_fb_sw_cloud.nc')

In [35]:
EBM_results_fb_SW_noncloud = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_SW_noncloud, models, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.856690e+02, 2.915912e+02, 3.028656e+02, 3.099837e+02, 3.077929e+02, 2.971487e+02, 2.919524e+02 K
EBM (perturbation) finished iteration 12000
2.868201e+02, 2.928055e+02, 3.040486e+02, 3.107644e+02, 3.088752e+02, 2.983099e+02, 2.931458e+02 K
EBM (perturbation) finished iteration 18000
2.875230e+02, 2.934996e+02, 3.046602e+02, 3.111448e+02, 3.094506e+02, 2.990222e+02, 2.939612e+02 K
EBM (perturbation) finished iteration 24000
2.879213e+02, 2.938832e+02, 3.049847e+02, 3.113437e+02, 3.097713e+02, 2.994449e+02, 2.944632e+02 K
EBM (perturbation) finished iteration 30000
2.881408e+02, 2.940927e+02, 3.051595e+02, 3.114515e+02, 3.099532e+02, 2.996919e+02, 2.947611e+02 K
EBM (perturbation) finished iteration 36000
2.882607e+02, 2.94206

In [36]:
EBM_results_fb_SW_noncloud.to_netcdf('nc_from_xarray/EBM_results_noQ_fb_sw_noncloud.nc')

In [37]:
EBM_results_fb_LW_wv = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_wv, models_noCNRM, D=0.96e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841682e+02, 2.896707e+02, 3.004603e+02, 3.080440e+02, 3.055918e+02, 2.953129e+02, 2.908846e+02 K
EBM (perturbation) finished iteration 6000
2.853178e+02, 2.910613e+02, 3.019250e+02, 3.092035e+02, 3.069836e+02, 2.966485e+02, 2.916399e+02 K
EBM (perturbation) finished iteration 12000
2.857590e+02, 2.914455e+02, 3.021528e+02, 3.093165e+02, 3.072191e+02, 2.970173e+02, 2.920792e+02 K
EBM (perturbation) finished iteration 18000
2.858804e+02, 2.915412e+02, 3.021996e+02, 3.093357e+02, 3.072704e+02, 2.971193e+02, 2.922192e+02 K
EBM (perturbation) finished iteration 24000
2.859111e+02, 2.915648e+02, 3.022105e+02, 3.093399e+02, 3.072834e+02, 2.971474e+02, 2.922593e+02 K
EBM (perturbation) finished iteration 30000
2.859187e+02, 2.915706e+02, 3.022132e+02, 3.093410e+02, 3.072869e+02, 2.971552e+02, 2.922704e+02 K
EBM (perturbation) finished iteration 36000
2.859206e+02, 2.91572

In [38]:
EBM_results_fb_LW_wv.to_netcdf('nc_from_xarray/EBM_results_noQ_fb_lw_wv.nc')

In [ ]:
EBM_results_fb_LW_ts = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ts, models, D=0.96e6, dt=5.0e-5, nmax=1e6)

AM2
Latitudes for test temperature printing:
-71.5
-53.5
-29.5
0.5
30.5
54.5
72.5
EBM (perturbation) finished iteration 0
2.841681e+02, 2.896705e+02, 3.004600e+02, 3.080436e+02, 3.055916e+02, 2.953127e+02, 2.908845e+02 K
EBM (perturbation) finished iteration 6000
2.851261e+02, 2.909820e+02, 3.022019e+02, 3.095000e+02, 3.072100e+02, 2.966000e+02, 2.914474e+02 K
EBM (perturbation) finished iteration 12000
2.862965e+02, 2.923440e+02, 3.036632e+02, 3.105113e+02, 3.086267e+02, 2.978853e+02, 2.925087e+02 K
EBM (perturbation) finished iteration 18000
2.875356e+02, 2.936645e+02, 3.049262e+02, 3.113306e+02, 3.098684e+02, 2.991894e+02, 2.937879e+02 K
EBM (perturbation) finished iteration 24000
2.887594e+02, 2.949085e+02, 3.060364e+02, 3.120246e+02, 3.109880e+02, 3.004810e+02, 2.951536e+02 K
EBM (perturbation) finished iteration 30000
2.899251e+02, 2.960626e+02, 3.070247e+02, 3.126310e+02, 3.120171e+02, 3.017392e+02, 2.965369e+02 K
EBM (perturbation) finished iteration 36000
2.910136e+02, 2.97124

In [ ]:
EBM_results_fb_LW_ts.to_netcdf('nc_from_xarray/EBM_results_noQ_fb_lw_ts.nc')

In [ ]:
#LW atmospheric temperature feedback: do just 10% case this time

In [ ]:

dt_dict_LW_ta_10p = {'AM2':             2.0e-5, 
                     'CAM3':            9.0e-5, 
                     'CAM4':            9.0e-5, 
                     'CNRM-AM6-DIA-v2': 3.0e-5, 
                     'ECHAM-6.1':       2.0e-5, 
                     'ECHAM-6.3':       2.0e-5, 
                     'IPSL-CM5A':       9.0e-5, 
                     'MIROC5':          9.0e-5, 
                     'MPAS':            9.0e-5, 
                     'MetUM-GA6-CTL':   2.0e-5, 
                     'MetUM-GA6-ENT':   2.0e-5, 
                     'NorESM2':         9.0e-5}
EBM_results_fb_LW_ta_10p = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_ta, models, 
                                              D=0.96e6, scale=.1, dt=dt_dict_LW_ta_10p, nmax=2e6)


In [ ]:
EBM_results_fb_LW_ta_10p.to_netcdf('nc_from_xarray/EBM_results_noQ_fb_lw_ta_10p.nc')

In [ ]:

#####   LW CLOUD ADJUSTMENT EXPERIMENTS   #####


In [ ]:
EBM_results_adj_LW_cloud = run_EBM_perturb_adj(ds_clim, ds_greg, da_adj_LW_cloud, models_10, D=0.96e6)

In [ ]:
EBM_results_fb_LW_cloud = run_EBM_perturb_fb(ds_clim, ds_greg, da_fb_LW_cloud, models_10, D = 0.96e6)

In [ ]:
EBM_results_adj_LW_cloud.to_netcdf('nc_from_xarray/EBM_results_noQ_adj_lw_cloud.nc')

In [ ]:
EBM_results_fb_LW_cloud.to_netcdf('nc_from_xarray/EBM_results_noQ_fb_lw_cloud.nc')